In [1]:
# Instalar paquetes
!pip install hubspot-api-client
!pip install rut_chile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 31.1 MB/s eta 0:00:00


In [ ]:
import requests
from rut_chile import rut_chile as rc
import pandas as pd
import numpy as np
import unicodedata
import re
import time
import psutil
import os

# Abrir archivo de log al inicio.
log_file = open("log.txt", "w")

log_file.write("====================================================\n")
log_file.write("== INICIO DEL SCRIPT DE CREACIÓN DE NUEVOS USUARIOS ==\n")
log_file.write("====================================================\n")

start = time.time()
process = psutil.Process(os.getpid())
mem_start = process.memory_info().rss



In [ ]:
# Función para validar rut
def validar_rut(rut):
    try:
        return rc.is_valid_rut(rut)
    except Exception as e:
        return False

def agregar_guion(valor):
    if pd.notna(valor) and len(valor) > 1:
        return valor[:-1] + '-' + valor[-1]
    else:
        return valor

def eliminar_cero(valor):
    if isinstance(valor, str) and valor.startswith('0'):
        return valor[1:]
    return valor

def consulta_api(id, access_token=None, identificador='rut'):
    # Definir URL y variables según identificador
    if identificador == 'rut':
        url = f'https://medicocontratosinfo.colegiomedico.cl/api/MedicoContratos/byRut/{id}/true'
    elif identificador == 'icm':
        url = f'https://medicocontratosinfo.colegiomedico.cl/api/MedicoContratos/byICM/{id}/true'
    elif identificador == 'direccion':
        url = f'https://medicocontratosinfo.colegiomedico.cl/api/Direcciones/byICM/{id}'
    elif identificador == 'datos_direccion':
        url = f'https://medicocontratosinfo.colegiomedico.cl/api/Direcc/byICM/{id}'

    # Obtener token si no se proporciona
    if access_token is None:
        access_token = autorizacion_api()

    # Configurar encabezados
    headers = {'Authorization': f'Bearer {access_token}'}

    try:
        # Realizar la solicitud
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Lanza una excepción para códigos HTTP >= 400

    except requests.exceptions.HTTPError as http_err:
        if response.status_code == 404:
            return {
                "error": "Recurso no encontrado.",
                "status": 404,
                "detalle": f"No se encontró el recurso solicitado con ID: {id}. URL: {url}"
            }
        elif response.status_code == 401:
            return {
                "error": "No autorizado.",
                "status": 401,
                "detalle": "El token es inválido o ha expirado. Intenta obtener un nuevo token."
            }
        else:
            return {
                "error": f"HTTP Error {response.status_code}",
                "detalle": response.text
            }
    except requests.exceptions.RequestException as req_err:
        return {
            "error": "Error de conexión.",
            "detalle": str(req_err)
        }

    # Si no hay errores, retorna el contenido como JSON
    return response.json()

def autorizacion_api():
    # URL de autenticación
    url_auth = 'https://authentication.colegiomedico.cl/api/authentication/authenticate'

    # Credenciales desde variables de entorno
    user = 'UsersCRSantiago'  # Valor por defecto
    psw = 'UsersCRSantiago2024_&_&'  # Valor por defecto

    # Verificar credenciales
    if not user or not psw:
        raise ValueError("Las credenciales de usuario o contraseña no están configuradas.")

    # Datos de autenticación
    data = {"userName": user, "password": psw}

    # Encabezados
    headers = {"Content-Type": "application/json"}

    # Solicitar token
    response_auth = requests.post(url_auth, headers=headers, json=data)

    # Verificar respuesta
    if response_auth.status_code != 200:
        raise Exception(f"Error al autenticar: {response_auth.status_code} - {response_auth.text}")

    # Retornar token
    access_token = response_auth.text
    return access_token.strip('"')

import pandas as pd

import pandas as pd

def obtener_datos_desde_api(id_inicial, id_final, identificador='icm'):
    resultados = []
    respuestas_vacias_consecutivas = 0
    limite_respuestas_vacias = 10

    for id_actual in range(id_inicial, id_final + 1):
        try:
            log_file.write(f"Consultando API externa - ID: {id_actual}\n")
            respuesta = consulta_api(str(id_actual), identificador=identificador)

            if isinstance(respuesta, dict) and 'status' in respuesta:
                status_code = respuesta.get('status', None)
                if status_code == 404:
                    log_file.write(f"ID {id_actual} no encontrado (404).\n")
                    respuestas_vacias_consecutivas += 1
                else:
                    log_file.write(f"ID {id_actual} devolvió un error ({status_code}).\n")
                    respuestas_vacias_consecutivas += 1
            elif not respuesta:
                log_file.write(f"ID {id_actual} no contiene información.\n")
                respuestas_vacias_consecutivas += 1
            else:
                resultados.append(respuesta)
                respuestas_vacias_consecutivas = 0

            if respuestas_vacias_consecutivas >= limite_respuestas_vacias:
                log_file.write(f"Se encontraron {limite_respuestas_vacias} respuestas vacías consecutivas. Deteniendo la ejecución.\n")
                break
        except Exception as e:
            log_file.write(f"Error con ID {id_actual}: {e}\n")
            continue
    return pd.DataFrame(resultados)
    
#recorrer la api del nacional desde un id_inical hasta un id_final
def obtener_datos_desde_api2(id_inicial, id_final, identificador='icm'):
    resultados = []

    for id_actual in range(id_inicial, id_final + 1):
        try:
            print(f"Consultando ID: {id_actual}")
            # Llama a la función consulta_api (definida previamente)
            respuesta = consulta_api(str(id_actual), identificador=identificador)
            resultados.append(respuesta)

        except Exception as e:
            print(f"Error con ID {id_actual}: {e}")
            continue

    # Convertir resultados en un DataFrame
    return pd.DataFrame(resultados)


def limpiar_valor(valor):
    """Elimina caracteres especiales, convierte a minúsculas, y elimina tildes"""
    # Eliminar tildes y acentos
    valor_sin_tildes = unicodedata.normalize('NFKD', valor).encode('ASCII', 'ignore').decode('ASCII')
    # Eliminar caracteres especiales
    valor_limpio = re.sub(r'[^\w\s]', '', valor_sin_tildes)
    return valor_limpio.replace(' ', '_').lower()


# **Obtener datos desde API NACIONAL**

In [ ]:
log_file.write("\n--- Bloque 1: Obteniendo datos desde API Nacional ---\n")
id_inicial = 59500
id_final = id_inicial + 10000
resultados_df = obtener_datos_desde_api(id_inicial, id_final)
log_file.write(f"Se obtuvieron {len(resultados_df)} registros desde la API Nacional.\n")



Consultando ID: 59504
ID 59504 no encontrado (404).
Consultando ID: 59505
ID 59505 no encontrado (404).
Consultando ID: 59506
ID 59506 no encontrado (404).
Consultando ID: 59507
ID 59507 no encontrado (404).
Consultando ID: 59508
ID 59508 no encontrado (404).
Consultando ID: 59509
ID 59509 no encontrado (404).
Consultando ID: 59510
ID 59510 no encontrado (404).
Consultando ID: 59511
ID 59511 no encontrado (404).
Consultando ID: 59512
ID 59512 no encontrado (404).
Consultando ID: 59513
ID 59513 no encontrado (404).

Se han encontrado 10 respuestas vacías consecutivas. Deteniendo la ejecución.


In [ ]:
# Convertir a int, reemplazando NaN con un valor específico (por ejemplo, 0)
log_file.write("\n--- Bloque 2: Procesando y limpiando datos ---\n")
resultados_df['rut'] = pd.to_numeric(resultados_df['rut'], errors='coerce').fillna(0).astype(int)
resultados_df['idMedico'] = pd.to_numeric(resultados_df['idMedico'], errors='coerce').fillna(0).astype(int)

In [ ]:
# Asegurar que 'rut' sea un entero eliminando decimales y convirtiendo a string
resultados_df['rut'] = resultados_df['rut'].astype('Int64').astype(str)
resultados_df['idMedico'] = resultados_df['idMedico'].astype('Int64').astype(str)
# Crear la columna con el formato correcto
resultados_df['rut_dv'] = resultados_df['rut'] + '-' + resultados_df['dvrut'].astype(str)

# Reemplazar 'k' por 'K' en rut
resultados_df['rut_dv'] = resultados_df['rut_dv'].str.replace('k', 'K')
#normalizar columna rut
resultados_df['rut_normalizado'] = (
    resultados_df['rut_dv'].str.replace(r'[\s.,-]', '', regex=True).apply(agregar_guion).apply(eliminar_cero).astype(str))
# Aplicar la función is_valid_rut y crear una nueva columna con los resultados
resultados_df['rut_valido'] = resultados_df['rut_normalizado'].apply(validar_rut)
# Filtrar las filas donde 'rut' es True
df_rut_filtrado = resultados_df[resultados_df['rut_valido'] == True]
log_file.write(f"Procesamiento finalizado. Se encontraron {len(df_rut_filtrado)} registros con RUT válido.\n")
#dataframe resultante
cant_registros = len(df_rut_filtrado)
cant_registros

134

# **Creación de nuevos usuarios en Hubspot**

In [ ]:
log_file.write("\n--- Bloque 3: Creando nuevos usuarios en HubSpot ---\n")

# Se lee la API Key desde los secrets
api_key = os.getenv('HUBSPOT_API_KEY')

url_search = 'https://api.hubapi.com/crm/v3/objects/contacts/search'
url_create = 'https://api.hubapi.com/crm/v3/objects/contacts'

headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json'
}

df = df_rut_filtrado
total_a_procesar = len(df)
log_file.write(f"Iniciando verificación y creación de {total_a_procesar} contactos...\n")

for index, row in df.iterrows():
    rut = row['rut_normalizado']
    search_data = {
        "filterGroups": [{"filters": [{"propertyName": "rut", "operator": "EQ", "value": rut}]}]
    }

    search_response = requests.post(url_search, headers=headers, json=search_data)

    if search_response.status_code == 200:
        search_result = search_response.json()
        if search_result['total'] == 0:
            properties = {
                "rut": row['rut_normalizado'],
                "firstname": row['nombre1'],
                "segundo_nombre": row['nombre2'],
                "lastname": row['apePat'],
                "apellido_materno": row['apeMat'],
                "rcm": row['idMedico']
            }
            data = {"properties": properties}
            create_response = requests.post(url_create, headers=headers, json=data)

            if create_response.status_code == 201:
                log_file.write(f"({index+1}/{total_a_procesar}) Éxito: Contacto con ID Medico {row['idMedico']} creado.\n")
            else:
                log_file.write(f"({index+1}/{total_a_procesar}) ERROR al crear contacto con RUT {rut}: {create_response.status_code} - {create_response.text}\n")
        else:
            log_file.write(f"({index+1}/{total_a_procesar}) Omitido: Contacto con RUT {rut} ya existe.\n")
    else:
        log_file.write(f"({index+1}/{total_a_procesar}) ERROR al buscar contacto con RUT {rut}: {search_response.status_code} - {search_response.text}\n")
    
    # --- ARREGLO: Pausa de medio segundo para evitar el Rate Limit ---
    time.sleep(0.5)

Contacto con RUT 19150868-8 ya existe
Contacto con RUT 19792718-6 ya existe
Contacto con RUT 19951495-4 ya existe
Contacto con RUT 19686654-K ya existe
Contacto con RUT 26202556-K ya existe
Contacto con RUT 19672972-0 ya existe
Contacto con RUT 19401307-8 ya existe
Contacto con RUT 26598154-2 ya existe
Contacto con RUT 18866230-7 ya existe
Contacto con RUT 26622152-5 ya existe
Contacto con RUT 19416194-8 ya existe
Contacto con RUT 18932427-8 ya existe
Contacto con RUT 27549450-K ya existe
Contacto con RUT 20104142-2 ya existe
Contacto con RUT 19891780-K ya existe
Error al buscar contacto con RUT 18214450-9: 429
{"status":"error","message":"You have reached your secondly limit.","errorType":"RATE_LIMIT","correlationId":"3dffaa99-1dfd-453d-9ab5-16d88a5f4dc8","policyName":"SECONDLY","groupName":"publicapi:crm:search:oauth:3628227:22173481"}
Contacto con RUT 27383700-0 ya existe
Contacto con RUT 14152573-5 ya existe
Contacto con RUT 28087182-6 ya existe
Contacto con RUT 20567616-3 ya exist

In [ ]:
mem_end = process.memory_info().rss
end = time.time()

log_file.write("\n====================================================\n")
log_file.write("== FIN DEL SCRIPT ==\n")
log_file.write(f"Tiempo total: {end - start:.2f} segundos\n")
log_file.write(f"Memoria total usada: {(mem_end - mem_start)/1024**2:.2f} MB\n")
log_file.write("====================================================\n")

# Se cierra el archivo para guardar los cambios.
log_file.close()

Tiempo total: 246.98 segundos
Memoria total usada: 4.19 MB
